In [ ]:
# from google.colab import drive
# drive.mount('/content/GoogleDrive')

In [1]:
!pip install split-folders

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from keras.models import Sequential, load_model
from keras.applications import ResNet50
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.utils.vis_utils import plot_model
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, zero_one_loss, classification_report
from sklearn.model_selection import train_test_split
from imutils import paths

import matplotlib.pyplot as plt
import seaborn as sb
import splitfolders
import cv2
import random

In [2]:
inputFolder = 'C:\\Users\\mooha\\jupyter\\Graduation Project\\image_datasets\\Brain Cancer'
outputFolder = 'C:\\Users\\mooha\\jupyter\\Graduation Project\\image_dataset_(Brain)'
# inputFolder = '/content/GoogleDrive/MyDrive/Colab Notebooks/Neural Networks/Graduation Project/Brain Cancer'
# outputFolder = '/content/GoogleDrive/MyDrive/Colab Notebooks/Neural Networks/Graduation Project/image_dataset_(Brain) '

In [ ]:
splitfolders.ratio(inputFolder, outputFolder, seed = 44, ratio = (0.8, 0.0, 0.2))

Copying files: 20000 files [01:19, 251.00 files/s] 


In [3]:
imgPaths = list(paths.list_images(outputFolder))
print(imgPaths[0].split('\\'))

random.shuffle(imgPaths)

classes = [] # classes = ['brain_menin', 'brain_glioma', 'brain_pituitary', 'no_tumor']
for imgPath in imgPaths:
  className = imgPath.split('\\')[-2]

  if className not in classes:
    classes.append(className)

print(classes)

X_train = []
y_train = []

X_test = []
y_test = []

IMGSIZE = 128
img_names = []

for imgPath in imgPaths:
  trainOrTest = imgPath.split('\\')[-3]
  className = imgPath.split('\\')[-2]

  img = cv2.imread(imgPath) #cv2.IMREAD_GRAYSCALE
  img = cv2.resize(img, (IMGSIZE,IMGSIZE))

  if trainOrTest == 'train':
    X_train.append(img)
    y_train.append(classes.index(className))
    img_names.append(imgPath.split('\\')[-1])

  elif trainOrTest == 'test':
    X_test.append(img)
    y_test.append(classes.index(className))

print(len(X_train), len(y_train))
print(len(X_test), len(y_test))

['C:', 'Users', 'mooha', 'jupyter', 'Graduation Project', 'image_dataset_(Brain)', 'test', 'brain_glioma', 'brain_glioma_0001.jpg']
['brain_pituitary', 'no_tumor', 'brain_menin', 'brain_glioma']
16000 16000
4000 4000


In [ ]:
df = pd.DataFrame({
    'File Name' : img_names,
    'Category' : y_train #y
})

# print(classes.index('brain_menin'))
# print(classes.index('brain_glioma'))
# print(classes.index('brain_pituitary'))
# print(classes.index('no_tumor'))

print(classes)
df.head()

In [ ]:
sb.countplot(x = y_train)

In [ ]:
sb.countplot(x = y_test)

In [7]:
X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [8]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(16000, 128, 128, 3) (16000,)
(4000, 128, 128, 3) (4000,)


In [10]:
input_shape = (IMGSIZE, IMGSIZE, 3) #tuple
num_classes = 4

# Load the pre-trained ResNet50 model (without the top classification layer)
base_model = ResNet50(weights = 'imagenet', include_top = False, input_shape = input_shape)

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

resnet_model = Sequential()

resnet_model.add(base_model)
resnet_model.add(Flatten())

resnet_model.add(Dense(256, activation = 'relu'))
resnet_model.add(Dropout(0.25))

resnet_model.add(Dense(128, activation = 'relu'))
resnet_model.add(Dropout(0.25))

resnet_model.add(Dense(64, activation = 'relu'))
resnet_model.add(Dense(num_classes, activation = 'softmax'))

resnet_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
results = resnet_model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 5, batch_size = 32, callbacks = [early_stopping]) 
# val_loss, val_acc = resnet_model.evaluate(X_test, y_test)

y_pred = resnet_model.predict(X_test)

resnet_model.summary()

94765736/94765736 [==============================] - 30s 0us/step
Epoch 1/5
500/500 [==============================] - 266s 525ms/step - loss: 0.6982 - accuracy: 0.8288 - val_loss: 0.1965 - val_accuracy: 0.9245
Epoch 2/5
500/500 [==============================] - 267s 534ms/step - loss: 0.2428 - accuracy: 0.9133 - val_loss: 0.1573 - val_accuracy: 0.9452
Epoch 3/5
500/500 [==============================] - 268s 536ms/step - loss: 0.1952 - accuracy: 0.9322 - val_loss: 0.1362 - val_accuracy: 0.9563
Epoch 4/5
500/500 [==============================] - 276s 552ms/step - loss: 0.1567 - accuracy: 0.9478 - val_loss: 0.1207 - val_accuracy: 0.9575
Epoch 5/5
125/125 [==============================] - 53s 409ms/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 f

In [2]:
classes = ['brain_menin', 'brain_glioma', 'brain_pituitary', 'no_tumor']
IMGSIZE = 128

In [3]:
# model.save('C:\\Users\\mooha\\jupyter\\Graduation Project\\model_image_(resNet).h5')
loaded_model = load_model('C:\\Users\\mooha\\jupyter\\Graduation Project\\model_image_(resNet).h5')

# model.save('/content/GoogleDrive/MyDrive/Colab Notebooks/Neural Networks/Graduation Project/model_image_(Brain).h5')
# loaded_model = model.load_weights('/content/GoogleDrive/MyDrive/Colab Notebooks/Neural Networks/Graduation Project/model_image_(Brain).h5')  

In [ ]:
plt.plot(results.history['accuracy'], label = 'Training Accuracy')
plt.plot(results.history['val_accuracy'], label = 'Testing Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(results.history['loss'], label = 'Training Loss')
plt.plot(results.history['val_loss'], label = 'Testing Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [13]:
print(classes)
print(y_test[:25], '\n')
print(y_pred[:25], '\n')

predict = []
for i in range(len(y_pred)):
  predict.append(np.argmax(y_pred[i]))

print(predict[:25], '\n')

cm = confusion_matrix(y_test, predict)
print(cm, '\n')

zol = zero_one_loss(y_test, predict, normalize = False)
print(zol, '\n')

cr = classification_report(y_test, predict)
print(cr)

['brain_pituitary', 'no_tumor', 'brain_menin', 'brain_glioma']
[0 2 2 2 1 2 1 1 1 0 2 3 2 3 1 1 0 0 1 0 2 0 1 3 2] 

[[6.70985162e-01 7.88721954e-05 3.28799695e-01 1.36322720e-04]
 [6.78911533e-07 4.73741046e-09 9.99280214e-01 7.19009433e-04]
 [5.80501514e-08 2.10330545e-10 9.99979198e-01 2.08011334e-05]
 [6.46960762e-05 7.53137442e-07 9.90081728e-01 9.85282566e-03]
 [1.58353796e-26 9.99999940e-01 6.76361636e-27 3.47100623e-25]
 [2.81022608e-07 1.20850283e-11 9.99969065e-01 3.05784961e-05]
 [8.33413916e-32 9.99999940e-01 2.39739543e-28 2.26754854e-29]
 [3.56471991e-34 9.99999940e-01 2.20899137e-33 3.25908908e-34]
 [9.27694366e-21 9.99999940e-01 1.63287066e-18 4.63699667e-19]
 [9.99924362e-01 1.66609018e-11 7.56077061e-05 1.15026304e-11]
 [2.49993708e-03 7.28131199e-05 9.66573179e-01 3.08541078e-02]
 [7.46239053e-08 5.35925911e-14 1.93771257e-08 9.99999821e-01]
 [7.21563725e-03 1.74557819e-04 8.87294412e-01 1.05315447e-01]
 [2.30758110e-06 1.61541104e-12 1.13827173e-06 9.99996483e-01]
 

In [ ]:
plt.figure(figsize = (5,5))
ConfusionMatrixDisplay(cm).plot()
plt.show()

In [ ]:
sb.countplot(x = predict)

In [ ]:
plot_model(model, show_shapes = True)

In [ ]:
test_img_path = 'C:\\Users\\mooha\\jupyter\\Graduation Project\\image_dataset_(Brain)\\test\\brain_glioma\\brain_glioma_4736.jpg'
# test_img_path = '/content/GoogleDrive/MyDrive/Colab Notebooks/Neural Networks/Graduation Project/image_dataset_(Brain)/test/brain_glioma/brain_glioma_4736.jpg'  
test_img = cv2.imread(test_img_path)
test_img = cv2.resize(test_img, (IMGSIZE,IMGSIZE))

plt.imshow(test_img)

test_img = test_img[tf.newaxis, ...]
test_predict = resnet_model.predict(test_img)
# test_predict = loaded_model(test_img)

str1 = test_img_path.split('\\')[-1].split('_')[0]
str2 = test_img_path.split('\\')[-1].split('_')[1]
str3 = classes[np.argmax(test_predict)].split('_')[0] + ' ' + classes[np.argmax(test_predict)].split('_')[1]

plt.title('Actual: ' + str1 + ' ' + str2 + '\n Predicted: ' + str3)

print(test_predict)
print(classes)
plt.show()